In [1]:
import warnings
warnings.simplefilter('ignore')
import json
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
import gc
from tqdm import tqdm

In [2]:
train_text_path = './data/annotations/labeled.json'
test_text_path = './data/annotations/test_a.json'
unlabeled_text_path = './data/annotations/unlabeled.json'
save_path = './temp_data/'

In [12]:
# 拼接title_ocr_asr
def load_text_data(file_path, file_type, save_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = json.load(f)
    df_text = pd.DataFrame(text)
    all_ocr_text = []
    all_time = []
    for ocr in tqdm(df_text['ocr']):
        all_ocr_text.append(', '.join([i['text'] for i in ocr]))
        all_time.append([i['time'] for i in ocr])
    df_text['ocr_text'] = all_ocr_text
    df_text['ocr_time'] = all_time
    df_text['text'] =  df_text['title'].astype(str) + "[SEP]" + \
                       df_text['asr'].str[0:60].astype(str) + df_text['asr'].str[-60:].astype(str) + \
                       df_text['ocr_text'].str[:60].astype(str) + df_text['ocr_text'].str[-60:].astype(str)
    if file_type == 'train':
        df_text = df_text[['id', 'text', 'category_id']]
    elif file_type == 'unlabeled' or file_type == 'test':
        df_text = df_text[['id', 'text']]
    df_text.to_pickle(save_path + file_type + '_text.pkl')
    print(len(df_text))
    gc.collect()

In [13]:
load_text_data(train_text_path, file_type='train', save_path=save_path)
load_text_data(test_text_path, file_type='test', save_path=save_path)
load_text_data(unlabeled_text_path, file_type='unlabeled', save_path=save_path)

100%|██████████| 100000/100000 [00:00<00:00, 581982.87it/s]


100000


100%|██████████| 25000/25000 [00:00<00:00, 533319.77it/s]


25000


100%|██████████| 1000000/1000000 [00:02<00:00, 355462.41it/s]


1000000


In [14]:
df = pd.read_pickle('./temp_data/test_text.pkl')

In [15]:
df

,id,text
0,13522617881232410931,#高铁 驶入驶出庆城县完整版视频[SEP]
1,13521771578659248229,我理解的保险经纪人，不只是为客户选购一款产品，申请一次核保，办理一次理赔。每一份保单背后都是...
2,13517541450239117642,哈哈哈哈哈#可爱的狗子[SEP]你看你看书吧。除了咱家你以为我讲你呀？你打我。哎呀。我打你你...
3,13521900652615370778,#瓷都品牌记 壹口烫面包为什么会成功？进来听完这四点，你有啥补充？@冷叔美食频道 [SEP]...
4,13510387983864567939,健身是一辈子要去做的事情。[SEP]嘎嘎。嘎嘎。
...,...,...
24995,13510025302058080277,头发细软贴头皮发量少这样剪减龄10岁#2021[SEP]那风采。那风采。
24996,13501164648885586259,#朋友圈背景图新年换个新背景新的一年好好工作努力赚钱，暴富暴美暴健康[SEP]2021全糖去...
24997,13513097487179651198,“二副明星团”团建[拥抱][拥抱][拥抱][SEP]嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯...
24998,13512339466130626703,做完视频，心里美美的哈[SEP]


In [ ]:
with open('./data/annotations/labeled.json', 'r', encoding='utf8') as f:
    a = json.load(f)


In [23]:
a

[{'id': '13477278056527828998',
  'category_id': '1203',
  'title': '管嘴管心，祸少福多;#国学禅语',
  'asr': '人多时管住嘴多错多是非多自找麻烦，人少时管住心。妄念多痛苦多，自寻烦恼。群处守住嘴，独处守住心。修己以清心为要，涉世以慎言为先。我是大悲关注我每天陪你保持好心态。',
  'ocr': [{'time': 0, 'text': '管嘴管心祸少福多人多时,管住嘴'}]},
 {'id': '13473068566164277577',
  'category_id': '2212',
  'title': '冬季羔羊严防这两种疾病',
  'asr': '很遥远的朋友大家好，今天和大家分享冬季高扬呢药房的两种疾病，第一个的是羔羊痢疾，第二个的是羔羊，肺炎啊，然后呢？这一大奖。每年了对羔羊用凉快意成都学啊，他只羔羊痢疾阳和意，五联疫苗接种。大家觉得我说的有帮助的话，给个关注，红星评论好么？',
  'ocr': [{'time': 0, 'text': '冬季羔羊严防这两种疾病Hello养羊的朋友大家好'}]},
 {'id': '13469963757121247370',
  'category_id': '2211',
  'title': '中国的世界第一，你知道有几个吗？#臭美哥自媒体 #微信热门话题 @臭美哥美食教程 @臭美哥创意时尚 双击视频给个小爱心哦！点@臭美哥自媒体 关注了解国内外新闻！',
  'asr': '五国的世界第一，你知道几个一中国人口世界第一，二粮食产量世界第一单高铁里程世界第一，自互联网用户世界第一波汽车制造，世界第一。六钢铁产量世界第一节，造船量世界第一，水电建设世界第一，就世界遗产总数世界。就第一师快递数量世界第一，11纺织品产量世界第一是黄金产量世界第一，13贸易总额世界第一，狮子手机产量世界第15核电技术世界第一厉害了我的国为祖国点赞！关注下期接着讲。',
  'ocr': [{'time': 0, 'text': '我国的世界第一十一:纺织品产量世界第一'}]},
 {'id': '13473722034684434511',
  'category_id': '1901',
  'title': '接到狙击任务，狙击对象竟是自己女朋